### Cleaning up Survey123 Geometry


* Monica's geojsons into our GCS?
* It would be great to save a parquet with one row per project element and a common project id.
* https://pypi.org/project/fs-gcsfs/
* Pip install `pip install fs-gcsfs` and `calitp_data_infra`

In [53]:
import geopandas as gpd
import pandas as pd
from shared_utils import utils

In [54]:
from calitp_data_infra import *
from calitp_data_analysis.sql import to_snakecase
from calitp_data_infra.storage import get_fs
fs = get_fs()
import os
import fsspec
from fs import open_fs
import _utils
import fiona
from fiona.io import ZipMemoryFile

In [55]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [56]:
#calitp-analytics-data/data-analyses/project_prioritization/Survey123_Geo
TCEP_SCCP_GCS = f"{_utils.GCS_FILE_PATH}Survey123_Geo/"

In [57]:
TCEP_SCCP_GCS

'gs://calitp-analytics-data/data-analyses/project_prioritization/Survey123_Geo/'

#### Geometry Intake geojson
* Same thing as layer 0 below

In [58]:
geo_path = "TCEP_SCCP_GeometryIntake_geojson_20230801.zip"

In [59]:
#with get_fs().open(f"{TCEP_SCCP_GCS}{geo_path}") as f:
    #tcep_sccp_geo = to_snakecase(gpd.read_file(f))
#

In [60]:
#tcep_sccp_geo.shape

In [61]:
#tcep_sccp_geo.project_name.nunique()

In [62]:
#tcep_sccp_geo.drop(columns = ['geometry'])

In [63]:
# tcep_sccp_geo.sort_values(by = ['project_name'])[['project_name','which_type_of_infrastructure_does_the_geometry_above_correspond_to?','geometry']]

In [64]:
# tcep_sccp_geo.explore(style_kwds = {'weight':5}, height = 400, width = 1000)

#### Geometry Scores
* https://stackoverflow.com/questions/64277987/python-geopandas-failing-to-read-misread-750mb-zip-esri-gdb-file-but-not-200mb
* https://fiona.readthedocs.io/en/latest/README.html
* https://fiona.readthedocs.io/en/stable/manual.html

In [109]:
def open_survey123():
    # https://gis.stackexchange.com/questions/255138/reading-the-names-of-geodatabase-file-layers-in-python
    file = "TCEP_SCCP_Score_Geometry_20230801.gdb.zip"
    fs.get(f'{TCEP_SCCP_GCS}{file}', 'tmp.gdb.zip')
    geo_layers = fiona.listlayers('tmp.gdb.zip')
    
    print(f"layers = {geo_layers}")
    
    gdf = pd.DataFrame()
    
    for i in geo_layers:
        # multiline = to_snakecase(gpd.read_file('tmp.gdb.zip', layer = geo_layers[0]))
        temp = to_snakecase(gpd.read_file('tmp.gdb.zip', layer = i))
        gdf = pd.concat([gdf, temp], axis=0)
        
    print("invalid geos")
    display(gdf[~gdf.geometry.is_valid][['projname','geometry','geopoint_comments']])
    
    gdf = gdf[gdf.geometry.is_valid].reset_index(drop = True)
    gdf = gdf.drop(columns = ['creationdate', 'editdate'])
    
    return gdf

In [110]:
all_results = open_survey123()

layers = ['TCEP_SCCP_GeometryIntake_All_Lns', 'TCEP_SCCP_GeometryIntake_All_Pts']
invalid geos


,projname,geometry,geopoint_comments
8,Stockton Channel Viaduct,None,NaN
9,Stockton Channel Viaduct,None,NaN
16,SR-46 East Antelope Grade Corridor Improvements,None,NaN


In [79]:
all_results.shape

(44, 17)

In [80]:
all_results.parentglobalid.nunique()

18

In [81]:
all_results.projname.nunique()

12

In [82]:
all_results.geopoint_type.value_counts()

Interchange Improvement    5
Bike/Pedestrian            4
Rail (Passenger)           3
Highway (Freight)          3
ITS                        3
Port/POE                   2
Highway                    1
Name: geopoint_type, dtype: int64

In [111]:
all_results.projname.unique()

array(['Watsonville-Santa Cruz Multimodal Corridor Program',
       'U.S. 101 Connected Communities Corridor Rail and Active Transportation Improvements',
       'Stockton Channel Viaduct',
       'SR 58 Mobility Improvements – Location 2',
       'The Desert Rail Infrastructure Improvement',
       'Westbound I-80 Cordelia Commercial Vehicle Enforcement Facility (WB I-80 CCVEF) Project',
       'Konocti Corridor - Segment 2B', 'Fix 5 Cascade Gateway',
       'Tulare Six-Lane and Paige Avenue Interchange Improvement Project',
       'I-80/San Pablo Dam Road Interchange Improvements - Phase 2',
       'Otay Mesa East Port of Entry',
       'National Highway Freight Network Improvement Program - State Route 47-Seaside Avenue & Navy Way Interchange Improvement Project'],
      dtype=object)

In [83]:
cols = ['parentglobalid','projname','geopoint_type','geopoint_type_existing','geopoint_comments','geometry']

In [84]:
all_results = all_results[all_results.geometry.is_valid]

In [85]:
all_results[cols].explore('projname', cmap='tab10', style_kwds = {'weight':5}, height = 400, width = 1000, legend = True)

In [86]:
# all_results.sort_values(by = ['projname'])[cols]

In [87]:
all_results.sort_values(by = ['projname']).drop(columns = ['geometry'])

,shape_length,parentglobalid,lyr_globalid,lyr,projname,lns,pts,ct_district,efis,ea,ppno,geopoint_type,geopoint_type_existing,geopoint_comments,creator,editor
43,NaN,{4D60FABF-CDFB-4C4A-870E-DC8F29664447},{EAE6FA43-F9DD-4673-A410-4A8267E1F53B},Pts2,Fix 5 Cascade Gateway,2,3,02,0215000083,0H920,3597,ITS,ITS,Providing,larissa.lee_caltrans,larissa.lee_caltrans
36,NaN,{4D60FABF-CDFB-4C4A-870E-DC8F29664447},{8AC36336-33EF-4D84-ACD5-A5F534060765},Pts3,Fix 5 Cascade Gateway,2,3,02,0215000083,0H920,3597,Highway (Freight),Highway (Freight),North Bound locations where Merging or weaving zone begins.,larissa.lee_caltrans,larissa.lee_caltrans
28,NaN,{C66E3299-BE33-46CD-83B7-D536F60E3E2A},{C66E3299-BE33-46CD-83B7-D536F60E3E2A},Pts1_Parent,Fix 5 Cascade Gateway,2,3,02,0215000083,0H920,3597,Highway (Freight),Highway (Freight),South Bound locations where Merging or weaving zone begins.,larissa.lee_caltrans,larissa.lee_caltrans
42,NaN,{4D60FABF-CDFB-4C4A-870E-DC8F29664447},{2AE7579E-05EC-49FC-9897-E4F125C27C60},Pts2,Fix 5 Cascade Gateway,2,3,02,0215000083,0H920,3597,ITS,ITS,Installing advance signing packages in both south and north bound.,larissa.lee_caltrans,larissa.lee_caltrans
16,10141.75,{4D60FABF-CDFB-4C4A-870E-DC8F29664447},{78B02F8B-958E-4C26-8B5F-D3EC606B09DD},Lns1,Fix 5 Cascade Gateway,2,3,02,0215000083,0H920,3597,NaN,Highway (Freight),North Bound Truck Only Lane,larissa.lee_caltrans,larissa.lee_caltrans
15,14979.18,{4D60FABF-CDFB-4C4A-870E-DC8F29664447},{5A0D0FD0-A4C1-4801-8320-DD7BFE229433},Lns1,Fix 5 Cascade Gateway,2,3,02,0215000083,0H920,3597,NaN,Highway (Freight),South Bound Truck Only Lane,larissa.lee_caltrans,larissa.lee_caltrans
22,1943.89,{A51B96AF-1ADD-46EE-87E1-D170733F8CDC},{792F83D9-1D88-4DA8-8628-B725988576E6},Lns1,I-80/San Pablo Dam Road Interchange Improvements - Phase 2,1,0,04,NaN,0A082,0242K,NaN,NaN,NaN,larissa.lee_caltrans,larissa.lee_caltrans
20,909.80,{A51B96AF-1ADD-46EE-87E1-D170733F8CDC},{24FB6AA0-85B8-4DA4-82EF-D0674430C8A6},Lns1,I-80/San Pablo Dam Road Interchange Improvements - Phase 2,1,0,04,NaN,0A082,0242K,NaN,NaN,NaN,larissa.lee_caltrans,larissa.lee_caltrans
21,757.36,{A51B96AF-1ADD-46EE-87E1-D170733F8CDC},{E3F1BC6A-EB32-4499-851E-685BB0EF16FD},Lns1,I-80/San Pablo Dam Road Interchange Improvements - Phase 2,1,0,04,NaN,0A082,0242K,NaN,NaN,NaN,larissa.lee_caltrans,larissa.lee_caltrans
14,6210.12,{7B1866C2-C823-465C-8018-82E621163252},{37E641AD-A93C-417F-8418-DE95E318A13C},Lns1,Konocti Corridor - Segment 2B,1,0,01,0118000079,29831,3121,NaN,NaN,NaN,larissa.lee_caltrans,larissa.lee_caltrans


In [88]:
#score = "TCEP_SCCP_Score_Geometry_20230801.gdb.zip"

In [89]:
#fs.get(f'{TCEP_SCCP_GCS}{score}', 'tmp.gdb.zip')

In [90]:
# geo_layers = fiona.listlayers('tmp.gdb.zip')

In [91]:
# geo_layers

In [92]:
# multiline = to_snakecase(gpd.read_file('tmp.gdb.zip', layer = geo_layers[0]))

In [93]:
# all_pts = to_snakecase(gpd.read_file('tmp.gdb.zip', layer = geo_layers[1]))

In [94]:
# all_pts.shape

In [95]:
# all_pts.parentglobalid.nunique()

In [96]:
# all_pts.sort_values(by = ['projname'])[all_pts_col]

In [97]:
# watsonville_bikeped = all_pts.loc[(all_pts.projname == "Watsonville-Santa Cruz Multimodal Corridor Program") & (all_pts.geopoint_type_existing == "Bike/Pedestrian")]

In [98]:
# watsonville_bikeped

In [99]:
# watsonville_bikeped[all_pts_col].explore(style_kwds = {'weight':5}, height = 400, width = 1000)

In [100]:
# multiline.shape

In [101]:
# multiline.projname.nunique()

In [102]:
# all_pts.projname.nunique()

In [103]:
#multiline2 = multiline[multiline.geometry.is_valid]

In [104]:
#multiline2.shape

In [105]:
#all_pts.columns

In [106]:
#ultiline2.columns

In [107]:
#multiline2.sort_values(by = ['projname'])[multiline_cols]

In [108]:
# multiline2[['projname','geometry','geopoint_comments']].explore('projname',cmap = 'tab10', style_kwds = {'weight':5}, height = 400, width = 1000)